Hago feature engineering en distintas etapa de la creación de mi obt, así que incorporo este proceso en un solo notebook. Decidi ir por obt ya que se me hizo más fácil visualizar la creación de mis variables y unirlas en una tabla grande al final. ¿Por qué no usé star o snowflake? No me parecía que tenía ningún tipo de utilidad tener las variables no cuantitativas (aparte del plan) para la predicción y tener tablas de dimensiones que no voy a ocupar me parecía una pérdida de tiempo y almacenamiento. Crearía un star, pero pedí permiso en el trabajo hasta las 12 y parece que voy a acabar 11:30 sin hacer eso.

In [202]:
import pandas as pd


In [203]:
dfCalls = pd.read_csv("../data/clean/clean_megaline_calls.csv")
dfInternet = pd.read_csv("../data/clean/clean_megaline_internet.csv")
dfMessages = pd.read_csv("../data/clean/clean_megaline_messages.csv")
dfPlans = pd.read_csv("../data/clean/clean_megaline_plans.csv")
dfUsers = pd.read_csv("../data/clean/clean_megaline_users.csv")

In [204]:
def get_billing_month(reg_date, call_date):
    months_since_reg = (call_date.year - reg_date.year) * 12 + (call_date.month - reg_date.month)
    billing_month_start = reg_date + pd.DateOffset(months=months_since_reg)
    return billing_month_start

# Feature engineering

In [205]:
#visualizando mis datos
print(dfCalls)
print(dfInternet)
print(dfMessages)
print(dfPlans)

        Unnamed: 0        id  user_id   call_date  duration
0                0   1000_93     1000  2018-12-27      8.52
1                1  1000_145     1000  2018-12-27     13.66
2                2  1000_247     1000  2018-12-27     14.48
3                3  1000_309     1000  2018-12-28      5.76
4                4  1000_380     1000  2018-12-30      4.22
...            ...       ...      ...         ...       ...
137730      137730  1499_199     1499  2018-11-21      8.72
137731      137731  1499_200     1499  2018-10-20     10.89
137732      137732  1499_201     1499  2018-09-21      8.12
137733      137733  1499_202     1499  2018-10-10      0.37
137734      137734  1499_203     1499  2018-12-29     13.86

[137735 rows x 5 columns]
        Unnamed: 0        id  user_id session_date  mb_used
0                0   1000_13     1000   2018-12-29    89.86
1                1  1000_204     1000   2018-12-31     0.00
2                2  1000_379     1000   2018-12-28   660.40
3            

Iniciemos con dfCalls, me interes obtener el valor mensual promedio que gasta cada persona.

In [206]:
#aegurandonos que esté en datetime
dfCalls['call_date'] = pd.to_datetime(dfCalls['call_date'])

In [207]:
#asegurandonos que esté en date time y haciendo que el plan tenga el mismo nombre que en el dfPlans
dfUsers['plan_name'] = dfUsers['plan']
dfUsers["reg_date"] = pd.to_datetime(dfUsers["reg_date"])

In [208]:
#haciendo el merge con la columna que creamos
dfPlans = dfPlans.merge(dfUsers[['user_id', 'plan_name','reg_date']], on='plan_name', how='inner')

In [209]:
#haciendo el merge para obtener los valores que vamos a utilizar para calcular los costos
dfCalls = dfCalls.merge(dfPlans[['user_id','minutes_included','usd_per_minute','reg_date']], on='user_id', how='inner')

In [210]:
#obteniendo el valor mensual de cada persona
dfCalls["billing_month"] = dfCalls.apply(lambda row: get_billing_month(row["reg_date"], row["call_date"]), axis=1)

df_agg = dfCalls.groupby(["user_id", "billing_month"]).agg(
    reg_date=("reg_date", "first"),
    total_minutes=("duration", "sum"),
    minutes_included=("minutes_included", "first"),
    usd_per_minute=("usd_per_minute", "first")
).reset_index()

df_agg["extra_minutes"] = np.maximum(df_agg["total_minutes"] - df_agg["minutes_included"], 0)
df_agg["extra_charges_minutes"] = df_agg["extra_minutes"] * df_agg["usd_per_minute"]


In [211]:
#visualizando el progreso
print(df_agg)

      user_id billing_month   reg_date  total_minutes  minutes_included  \
0        1000    2018-12-24 2018-12-24         116.83              3000   
1        1001    2018-08-13 2018-08-13         171.14               500   
2        1001    2018-09-13 2018-08-13         297.69               500   
3        1001    2018-10-13 2018-08-13         374.11               500   
4        1001    2018-11-13 2018-08-13         404.59               500   
...       ...           ...        ...            ...               ...   
2253     1498    2018-12-04 2018-02-04         324.77               500   
2254     1499    2018-09-06 2018-05-06         330.37               500   
2255     1499    2018-10-06 2018-05-06         363.28               500   
2256     1499    2018-11-06 2018-05-06         288.56               500   
2257     1499    2018-12-06 2018-05-06         468.10               500   

      usd_per_minute  extra_minutes  extra_charges_minutes  
0               0.01            0.0   

In [212]:
#comprobando que haya gente que se exceda
df_agg[df_agg.loc[:,'extra_charges_minutes'] > 0]

,user_id,billing_month,reg_date,total_minutes,minutes_included,usd_per_minute,extra_minutes,extra_charges_minutes
9,1003,2018-12-28,2018-01-28,1041.00,500,0.03,541.00,16.2300
23,1007,2018-10-11,2018-07-11,610.65,500,0.03,110.65,3.3195
25,1007,2018-12-11,2018-07-11,579.43,500,0.03,79.43,2.3829
30,1009,2018-06-22,2018-04-22,775.52,500,0.03,275.52,8.2656
31,1009,2018-07-22,2018-04-22,826.74,500,0.03,326.74,9.8022
...,...,...,...,...,...,...,...,...
2197,1488,2018-11-15,2018-05-15,751.66,500,0.03,251.66,7.5498
2198,1488,2018-12-15,2018-05-15,678.14,500,0.03,178.14,5.3442
2215,1491,2018-10-27,2018-04-27,551.89,500,0.03,51.89,1.5567
2230,1494,2018-10-19,2018-04-19,585.82,500,0.03,85.82,2.5746


Bueno, ahora es hora de obtener un total y un promedio haciendo group por user_id y ya estamos

In [213]:
#obteniendo el total y el promedio. Visualizando al final
df_group = df_agg.groupby('user_id').agg(
    extra_minutes=('extra_minutes', 'sum'),
    extra_charges_minutes =('extra_charges_minutes', 'sum'),
    avg_extra_charges_per_month_minutes = ('extra_charges_minutes', 'mean')
).reset_index()

print(df_group)

     user_id  extra_minutes  extra_charges_minutes  \
0       1000           0.00                 0.0000   
1       1001           0.00                 0.0000   
2       1002           0.00                 0.0000   
3       1003         541.00                16.2300   
4       1004           0.00                 0.0000   
..       ...            ...                    ...   
476     1495          78.03                 2.3409   
477     1496           0.00                 0.0000   
478     1497           0.00                 0.0000   
479     1498           0.00                 0.0000   
480     1499           0.00                 0.0000   

     avg_extra_charges_per_month_minutes  
0                               0.000000  
1                               0.000000  
2                               0.000000  
3                              16.230000  
4                               0.000000  
..                                   ...  
476                             0.585225  
477    

In [214]:
#obteniendo más métricas que nos podrían resultar útiles
df_calls_total = dfCalls.groupby('user_id').agg(
    latest = ('call_date','max'),
    total_calls=('user_id', 'size'),        
    total_minutes=('duration', 'sum')
).reset_index()

df_calls_total['promedio_minutes_por_ocasion'] = df_calls_total['total_minutes'] / df_calls_total['total_calls']

df_calls_total = df_calls_total.merge(df_group,on = 'user_id',how='inner')

Agreguemos un promedio por mes, el total de cuanto ha pagado por su plan se puede agregar al final.

In [215]:
#visualizando nuestro df
print(df_calls_total)

     user_id     latest  total_calls  total_minutes  \
0       1000 2018-12-31           16         116.83   
1       1001 2018-12-31          261        1640.46   
2       1002 2018-12-31          113         777.13   
3       1003 2018-12-31          149        1041.00   
4       1004 2018-12-30          370        2618.95   
..       ...        ...          ...            ...   
476     1495 2018-12-31          253        1666.41   
477     1496 2018-12-31          195        1376.21   
478     1497 2018-12-31           54         276.53   
479     1498 2018-12-31          451        3029.97   
480     1499 2018-12-31          204        1450.31   

     promedio_minutes_por_ocasion  extra_minutes  extra_charges_minutes  \
0                        7.301875           0.00                 0.0000   
1                        6.285287           0.00                 0.0000   
2                        6.877257           0.00                 0.0000   
3                        6.986577      

In [216]:
#visualizando el contenido
print(dfCalls.nunique())


Unnamed: 0          137735
id                  137735
user_id                481
call_date              351
duration              2802
minutes_included         2
usd_per_minute           2
reg_date               259
billing_month          307
dtype: int64


Vemos que solo hay 481 user_id, para los que no esten ponemos 0 ya que no han consumido esta área. Esto se hará después del merge

In [217]:
#visualizando
print(df_calls_total)

     user_id     latest  total_calls  total_minutes  \
0       1000 2018-12-31           16         116.83   
1       1001 2018-12-31          261        1640.46   
2       1002 2018-12-31          113         777.13   
3       1003 2018-12-31          149        1041.00   
4       1004 2018-12-30          370        2618.95   
..       ...        ...          ...            ...   
476     1495 2018-12-31          253        1666.41   
477     1496 2018-12-31          195        1376.21   
478     1497 2018-12-31           54         276.53   
479     1498 2018-12-31          451        3029.97   
480     1499 2018-12-31          204        1450.31   

     promedio_minutes_por_ocasion  extra_minutes  extra_charges_minutes  \
0                        7.301875           0.00                 0.0000   
1                        6.285287           0.00                 0.0000   
2                        6.877257           0.00                 0.0000   
3                        6.986577      

Ahora vamos a obetener los precios de dfInternet

In [218]:
#viendo el contenido
print(dfInternet.nunique())

Unnamed: 0      104825
id              104825
user_id            489
session_date       351
mb_used          57624
dtype: int64


In [219]:
#asegurandonos que este en datetime
dfInternet['session_date'] = pd.to_datetime(dfInternet['session_date'])

In [220]:
#cambiando para posiblemente utiliza mb en lugar de gb (hice esto pero me parece que es mejor ir con gb)
dfPlans['usd_per_mb'] = dfPlans['usd_per_gb'].astype(float) / 1000

In [221]:
#visualizando
print(dfPlans)

     Unnamed: 0  messages_included  mb_per_month_included  minutes_included  \
0             0                 50                  15360               500   
1             0                 50                  15360               500   
2             0                 50                  15360               500   
3             0                 50                  15360               500   
4             0                 50                  15360               500   
..          ...                ...                    ...               ...   
495           1               1000                  30720              3000   
496           1               1000                  30720              3000   
497           1               1000                  30720              3000   
498           1               1000                  30720              3000   
499           1               1000                  30720              3000   

     usd_monthly_pay  usd_per_gb  usd_per_message  

In [222]:
#visualizando
print(dfInternet)

        Unnamed: 0        id  user_id session_date  mb_used
0                0   1000_13     1000   2018-12-29    89.86
1                1  1000_204     1000   2018-12-31     0.00
2                2  1000_379     1000   2018-12-28   660.40
3                3  1000_413     1000   2018-12-26   270.99
4                4  1000_442     1000   2018-12-27   880.22
...            ...       ...      ...          ...      ...
104820      104820  1499_215     1499   2018-10-20   218.06
104821      104821  1499_216     1499   2018-12-30   304.72
104822      104822  1499_217     1499   2018-09-22   292.75
104823      104823  1499_218     1499   2018-12-07     0.00
104824      104824  1499_219     1499   2018-12-24   758.31

[104825 rows x 5 columns]


In [223]:
#uniendo los datos para obtener el valor y asi sacar el costo
dfInternet = dfInternet.merge(dfPlans[['user_id','mb_per_month_included','usd_per_gb','reg_date']], on='user_id', how='inner')

In [224]:
#visualizando
print(dfInternet)

        Unnamed: 0        id  user_id session_date  mb_used  \
0                0   1000_13     1000   2018-12-29    89.86   
1                1  1000_204     1000   2018-12-31     0.00   
2                2  1000_379     1000   2018-12-28   660.40   
3                3  1000_413     1000   2018-12-26   270.99   
4                4  1000_442     1000   2018-12-27   880.22   
...            ...       ...      ...          ...      ...   
104820      104820  1499_215     1499   2018-10-20   218.06   
104821      104821  1499_216     1499   2018-12-30   304.72   
104822      104822  1499_217     1499   2018-09-22   292.75   
104823      104823  1499_218     1499   2018-12-07     0.00   
104824      104824  1499_219     1499   2018-12-24   758.31   

        mb_per_month_included  usd_per_gb   reg_date  
0                       30720           7 2018-12-24  
1                       30720           7 2018-12-24  
2                       30720           7 2018-12-24  
3                      

In [225]:
#obteniendo más métricas que nos podrían resultar útiles
dfInternet["billing_month"] = dfInternet.apply(lambda row: get_billing_month(row["reg_date"], row["session_date"]), axis=1)

df_agg = dfInternet.groupby(["user_id", "billing_month"]).agg(
    reg_date=("reg_date", "first"),
    total_mb=("mb_used", "sum"),
    mb_included=("mb_per_month_included", "first"),
    usd_per_gb=("usd_per_gb", "first")
).reset_index()

df_agg["extra_mb"] = np.maximum(df_agg["total_mb"] - df_agg["mb_included"], 0)


df_agg["extra_charges_gb"] = (np.ceil(df_agg["extra_mb"] / 1000) * df_agg["usd_per_gb"])


In [226]:
#revisando que haya gente que se haya exedido
df_agg[df_agg.loc[:,'extra_charges_gb'] > 0]

,user_id,billing_month,reg_date,total_mb,mb_included,usd_per_gb,extra_mb,extra_charges_gb
3,1001,2018-10-13,2018-08-13,22330.49,15360,10,6970.49,70.0
4,1001,2018-11-13,2018-08-13,18504.30,15360,10,3144.30,40.0
5,1001,2018-12-13,2018-08-13,19369.18,15360,10,4009.18,50.0
7,1002,2018-11-21,2018-10-21,19345.08,15360,10,3985.08,40.0
9,1003,2018-12-28,2018-01-28,27044.14,15360,10,11684.14,120.0
...,...,...,...,...,...,...,...,...
2271,1498,2018-11-04,2018-02-04,19168.55,15360,10,3808.55,40.0
2272,1498,2018-12-04,2018-02-04,23137.69,15360,10,7777.69,80.0
2274,1499,2018-10-06,2018-05-06,19492.43,15360,10,4132.43,50.0
2275,1499,2018-11-06,2018-05-06,16813.83,15360,10,1453.83,20.0


Hay gente que se a excedido, ahora es hora de obtener un total haciendo group por user_id y ya estamos

In [227]:
#agrupando y obteniendo valores que me interesan
df_group = df_agg.groupby('user_id').agg(
    extra_mb=('extra_mb', 'sum'),
    extra_charges_gb =('extra_charges_gb', 'sum'),
    avg_extra_charges_per_month_gb = ('extra_charges_gb', 'mean')

).reset_index()

df_group

,user_id,extra_mb,extra_charges_gb,avg_extra_charges_per_month_gb
0,1000,0.00,0.0,0.000000
1,1001,14123.97,160.0,32.000000
2,1002,3985.08,40.0,13.333333
3,1003,11684.14,120.0,120.000000
4,1004,43103.97,470.0,58.750000
...,...,...,...,...
484,1495,37450.96,390.0,97.500000
485,1496,2108.26,40.0,8.000000
486,1497,0.00,0.0,0.000000
487,1498,59117.95,640.0,58.181818


In [228]:
#obteniendo la tabla final de internet que vamo a utiliza
df_internet_total = dfInternet.groupby('user_id').agg(
    total_internet=('user_id', 'size'),        
    total_mb=('mb_used', 'sum'),
    latest = ('session_date','max')
).reset_index()

df_internet_total['promedio_mb_por_ocasion'] = df_internet_total['total_mb'] / df_internet_total['total_internet']

df_internet_total = df_internet_total.merge(df_group,on = 'user_id',how='inner')

In [229]:
#visualizando
print(df_internet_total)

     user_id  total_internet   total_mb     latest  promedio_mb_por_ocasion  \
0       1000               5    1901.47 2018-12-31               380.294000   
1       1001             245   80437.94 2018-12-31               328.318122   
2       1002             124   40293.33 2018-12-31               324.946210   
3       1003              52   27044.14 2018-12-31               520.079615   
4       1004             460  156352.81 2018-12-31               339.897413   
..       ...             ...        ...        ...                      ...   
484     1495             290   98890.96 2018-12-31               341.003310   
485     1496             225   64268.64 2018-12-31               285.638400   
486     1497              31   11106.55 2018-12-30               358.275806   
487     1498             657  227525.13 2018-12-31               346.309178   
488     1499             220   71350.23 2018-12-31               324.319227   

     extra_mb  extra_charges_gb  avg_extra_charges_

Ahora vamos por dfMessages

In [230]:
#visualizando
print(dfMessages.nunique())

Unnamed: 0      76051
id              76051
user_id           402
message_date      351
dtype: int64


In [231]:
#asegurandonos que esten en datetime
dfMessages['message_date'] = pd.to_datetime(dfMessages['message_date'])

In [232]:
#uniendo para obtener el costo
dfMessages = dfMessages.merge(dfPlans[['user_id','messages_included','usd_per_message','reg_date']], on='user_id', how='inner')

In [233]:
#obteniendo los valores que nos interesan
dfMessages["billing_month"] = dfMessages.apply(lambda row: get_billing_month(row["reg_date"], row["message_date"]), axis=1)

df_agg = dfMessages.groupby(["user_id", "billing_month"]).agg(
    reg_date=("reg_date", "first"),
    total_messages=("message_date", "count"),
    messages_included=("messages_included", "first"),
    usd_per_message=("usd_per_message", "first")
).reset_index()

df_agg["extra_messages"] = np.maximum(df_agg["total_messages"] - df_agg["messages_included"], 0)
df_agg["extra_charges_messages"] = df_agg["extra_messages"] * df_agg["usd_per_message"]

In [234]:
#viendo si gente se ha excedido
print(df_agg[df_agg.loc[:,'extra_charges_messages'] > 0])

      user_id billing_month   reg_date  total_messages  messages_included  \
3        1001    2018-10-13 2018-08-13              53                 50   
21       1007    2018-08-11 2018-07-11              51                 50   
23       1007    2018-10-11 2018-07-11              59                 50   
38       1014    2018-12-25 2018-11-25              64                 50   
39       1015    2018-12-05 2018-12-05              68                 50   
...       ...           ...        ...             ...                ...   
1779     1489    2018-11-24 2018-05-24              91                 50   
1780     1489    2018-12-24 2018-05-24              86                 50   
1783     1491    2018-06-27 2018-04-27              54                 50   
1784     1491    2018-07-27 2018-04-27              64                 50   
1787     1491    2018-10-27 2018-04-27              51                 50   

      usd_per_message  extra_messages  extra_charges_messages  
3          

Hay gente que se ha excedido, ahora es hora de obtener un total haciendo group por user_id y ya estamos

In [235]:
#obteniendo columnas sobre el extra individual y visualizando
df_group = df_agg.groupby('user_id').agg(
    extra_messages=('extra_messages', 'sum'),
    extra_charges_messages =('extra_charges_messages', 'sum'),
    avg_extra_charges_per_month_messages = ('extra_charges_messages', 'mean')

).reset_index()

print(df_group)

     user_id  extra_messages  extra_charges_messages  \
0       1000               0                    0.00   
1       1001               3                    0.09   
2       1002               0                    0.00   
3       1003               0                    0.00   
4       1004               0                    0.00   
..       ...             ...                     ...   
397     1491              19                    0.57   
398     1492               0                    0.00   
399     1494               0                    0.00   
400     1496               0                    0.00   
401     1497               0                    0.00   

     avg_extra_charges_per_month_messages  
0                                0.000000  
1                                0.018000  
2                                0.000000  
3                                0.000000  
4                                0.000000  
..                                    ...  
397                

In [236]:
#visualizando
print(dfMessages)

       Unnamed: 0        id  user_id message_date  messages_included  \
0               0  1000_125     1000   2018-12-27               1000   
1               1  1000_160     1000   2018-12-31               1000   
2               2  1000_223     1000   2018-12-31               1000   
3               3  1000_251     1000   2018-12-27               1000   
4               4  1000_255     1000   2018-12-26               1000   
...           ...       ...      ...          ...                ...   
76046       76046  1497_526     1497   2018-12-24               1000   
76047       76047  1497_536     1497   2018-12-24               1000   
76048       76048  1497_547     1497   2018-12-31               1000   
76049       76049  1497_558     1497   2018-12-24               1000   
76050       76050  1497_613     1497   2018-12-23               1000   

       usd_per_message   reg_date billing_month  
0                 0.01 2018-12-24    2018-12-24  
1                 0.01 2018-12-24  

In [237]:
#obteniendo la tabla final que vamos a utilizar
df_messages_total = dfMessages.groupby('user_id').agg(
    total_messages=('message_date', 'count'),
    latest = ('message_date','max')
).reset_index()

df_messages_total = df_messages_total.merge(df_group,on = 'user_id',how='inner')

In [238]:
#visualizando
print(df_messages_total)

     user_id  total_messages     latest  extra_messages  \
0       1000              11 2018-12-31               0   
1       1001             207 2018-12-30               3   
2       1002              88 2018-12-31               0   
3       1003              50 2018-12-31               0   
4       1004             177 2018-12-30               0   
..       ...             ...        ...             ...   
397     1491             409 2018-12-31              19   
398     1492             108 2018-12-31               0   
399     1494             174 2018-12-31               0   
400     1496              65 2018-12-30               0   
401     1497              50 2018-12-31               0   

     extra_charges_messages  avg_extra_charges_per_month_messages  
0                      0.00                              0.000000  
1                      0.09                              0.018000  
2                      0.00                              0.000000  
3                  

In [239]:
#obteniendo la union de calls internet y messages. Usando outer para que entren los que estan en una pero no en otra
df_ci = df_calls_total.merge(df_internet_total,how='outer',on = 'user_id')

df_ci['latest'] = df_ci[['latest_x', 'latest_y']].max(axis=1)

df_ci = df_ci.drop(columns=['latest_x', 'latest_y'])

df_cim = df_ci.merge(df_messages_total,how='outer',on = 'user_id')

df_cim['latest'] = df_cim[['latest_x', 'latest_y']].max(axis=1)

df_cim = df_cim.drop(columns=['latest_x', 'latest_y'])


In [240]:
#visualizando
print(df_cim)

     user_id  total_calls  total_minutes  promedio_minutes_por_ocasion  \
0       1000         16.0         116.83                      7.301875   
1       1001        261.0        1640.46                      6.285287   
2       1002        113.0         777.13                      6.877257   
3       1003        149.0        1041.00                      6.986577   
4       1004        370.0        2618.95                      7.078243   
..       ...          ...            ...                           ...   
485     1495        253.0        1666.41                      6.586601   
486     1496        195.0        1376.21                      7.057487   
487     1497         54.0         276.53                      5.120926   
488     1498        451.0        3029.97                      6.718337   
489     1499        204.0        1450.31                      7.109363   

     extra_minutes  extra_charges_minutes  \
0             0.00                 0.0000   
1             0.00   

In [241]:
#primero hallar cual de los 3 latest es el mayor y de ahi hacer el fill_na
df_cim.fillna(0,inplace=True) 

In [242]:
#viendo cuanta gente hay
print(df_cim.shape)

(490, 18)


In [243]:
#visualizando
print(df_cim)

     user_id  total_calls  total_minutes  promedio_minutes_por_ocasion  \
0       1000         16.0         116.83                      7.301875   
1       1001        261.0        1640.46                      6.285287   
2       1002        113.0         777.13                      6.877257   
3       1003        149.0        1041.00                      6.986577   
4       1004        370.0        2618.95                      7.078243   
..       ...          ...            ...                           ...   
485     1495        253.0        1666.41                      6.586601   
486     1496        195.0        1376.21                      7.057487   
487     1497         54.0         276.53                      5.120926   
488     1498        451.0        3029.97                      6.718337   
489     1499        204.0        1450.31                      7.109363   

     extra_minutes  extra_charges_minutes  \
0             0.00                 0.0000   
1             0.00   

Ahora tenemos que agregar el costo por cada uno y el costo total. Para esto unamosle ya a una tabla grande y vamos botando columnas

In [244]:
#esta es nuestra obt
df = df_cim.merge(dfUsers,how='outer',on = 'user_id')

In [245]:
#botando las columnas que no nos sirven
df.drop(['first_name', 'last_name', 'city','plan'], axis=1, inplace=True, errors='ignore')

In [246]:
#visualizando
df.shape

(500, 22)

Agregamos months, days y hacemos dummies.

In [247]:
#agregando months y days
df["reg_date"] = pd.to_datetime(df["reg_date"], errors='coerce')
df["latest"] = pd.to_datetime(df["latest"], errors='coerce')

df["days_since_reg"] = (df["latest"] - df["reg_date"]).dt.days

df["months_since_reg"] = (df["latest"].dt.year - df["reg_date"].dt.year) * 12 + (df["latest"].dt.month - df["reg_date"].dt.month)

In [248]:
#visualizando
print(df)

     user_id  total_calls  total_minutes  promedio_minutes_por_ocasion  \
0       1000         16.0         116.83                      7.301875   
1       1001        261.0        1640.46                      6.285287   
2       1002        113.0         777.13                      6.877257   
3       1003        149.0        1041.00                      6.986577   
4       1004        370.0        2618.95                      7.078243   
..       ...          ...            ...                           ...   
495     1495        253.0        1666.41                      6.586601   
496     1496        195.0        1376.21                      7.057487   
497     1497         54.0         276.53                      5.120926   
498     1498        451.0        3029.97                      6.718337   
499     1499        204.0        1450.31                      7.109363   

     extra_minutes  extra_charges_minutes  \
0             0.00                 0.0000   
1             0.00   

In [249]:
#haciendo dummies de plan_name
df = pd.get_dummies(df, columns=['plan_name'])

Ahora hacemos total de exceso, total de exceso promedio

In [250]:
#creando el total que se ha excedido
df['total_exceso'] = df['extra_charges_messages'] + df['extra_charges_minutes'] + df['extra_charges_gb']

In [251]:
#creando el promedio mensual total con el que cada persona se ha excedido
df['avg_extra_charges_per_month'] = df['avg_extra_charges_per_month_messages'] + df['avg_extra_charges_per_month_minutes'] + df['avg_extra_charges_per_month_gb']

In [252]:
#botando una columna que no nos sirve
df.drop(['latest','reg_date'],inplace=True, axis =1)

In [253]:
#visualizando
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 25 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   user_id                               500 non-null    int64  
 1   total_calls                           490 non-null    float64
 2   total_minutes                         490 non-null    float64
 3   promedio_minutes_por_ocasion          490 non-null    float64
 4   extra_minutes                         490 non-null    float64
 5   extra_charges_minutes                 490 non-null    float64
 6   avg_extra_charges_per_month_minutes   490 non-null    float64
 7   total_internet                        490 non-null    float64
 8   total_mb                              490 non-null    float64
 9   promedio_mb_por_ocasion               490 non-null    float64
 10  extra_mb                              490 non-null    float64
 11  extra_charges_gb   

In [254]:
#hay que rellenar con 10 ya que no todos participan en todo
df.fillna(0,inplace=True)

In [255]:
#+1 pq pagas desde el mes 0
df['total_pagos_mensuales'] = (df['months_since_reg'] + 1) * df['plan_name_surf'].apply(lambda x: 20 if x else 70)


In [256]:
#obteniendo el total de pagos y el total de pagos promedio
df['total_pagos'] = df['total_pagos_mensuales'] + df['total_exceso']
df['total_pagos_promedios'] = df['avg_extra_charges_per_month'] + df['plan_name_surf'].apply(lambda x: 20 if x else 70)


In [257]:
#visualizando
print(df)


     user_id  total_calls  total_minutes  promedio_minutes_por_ocasion  \
0       1000         16.0         116.83                      7.301875   
1       1001        261.0        1640.46                      6.285287   
2       1002        113.0         777.13                      6.877257   
3       1003        149.0        1041.00                      6.986577   
4       1004        370.0        2618.95                      7.078243   
..       ...          ...            ...                           ...   
495     1495        253.0        1666.41                      6.586601   
496     1496        195.0        1376.21                      7.057487   
497     1497         54.0         276.53                      5.120926   
498     1498        451.0        3029.97                      6.718337   
499     1499        204.0        1450.31                      7.109363   

     extra_minutes  extra_charges_minutes  \
0             0.00                 0.0000   
1             0.00   

Nuestra columna a predecir es total_pagos_promedios. El modelo va a aprender que esta es derivada de las columnas de las cuales es derivada así que tenemos que dar de baja esas columnas.

Vamos a dar de baja a todas las columnas que tengan que ver con plata. Tambien el user id

In [258]:
df.drop(['total_pagos_mensuales','total_pagos','avg_extra_charges_per_month','avg_extra_charges_per_month_minutes','avg_extra_charges_per_month_gb','user_id'], axis = 1, inplace = True)

In [259]:
#visualizando
print(df)
#la obt no contiene todos los valores de todos los csv pero si contiene los valores que voy a usar para ml

     total_calls  total_minutes  promedio_minutes_por_ocasion  extra_minutes  \
0           16.0         116.83                      7.301875           0.00   
1          261.0        1640.46                      6.285287           0.00   
2          113.0         777.13                      6.877257           0.00   
3          149.0        1041.00                      6.986577         541.00   
4          370.0        2618.95                      7.078243           0.00   
..           ...            ...                           ...            ...   
495        253.0        1666.41                      6.586601          78.03   
496        195.0        1376.21                      7.057487           0.00   
497         54.0         276.53                      5.120926           0.00   
498        451.0        3029.97                      6.718337           0.00   
499        204.0        1450.31                      7.109363           0.00   

     extra_charges_minutes  total_inter

Ahora solo queda exportar

In [261]:
df.to_csv("../data/ml/ml_ready.csv")